In [3]:
from tensorflow.keras.models import load_model
import numpy as np
import os
import sys
import pathlib
from pathlib import Path

2024-05-07 19:05:18.976660: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 19:05:19.018242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 19:05:19.650022: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent
directory_path = parent_dir / 'dataset'
import swallowing_recognition

In [5]:
from swallowing_recognition import dataset
from swallowing_recognition.variable_data_set import VariableDataSet
from swallowing_recognition.mlp import MLP
from swallowing_recognition.cnn import CNN

# データを準備

In [6]:
name = "haruki"

In [7]:
prediction_swallowing_folder = directory_path / name / 'swallowing'
prediction_voice_folder = directory_path / name / 'voice'
prediction_cough_folder = directory_path / name / 'cough'
prediction_hitting_folder = directory_path / name / 'hitting'    
prediction_movement_folder = directory_path / name / 'movement'    

# DenceNetで予測する

In [8]:
# loaded_model = load_model('20240220_binary_ep50_bs32_v2.keras')
loaded_model = load_model('except_haruki20240507_binary_ep40_bs32_v1.keras')

2024-05-07 19:06:20.031335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22051 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [9]:
prediction_data = dataset.DataSet(50, 224, 224, 3, 2)
prediction_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), 0)
prediction_data.folder_to_dataset(prediction_voice_folder, np.array(1), 10)
prediction_data.folder_to_dataset(prediction_cough_folder, np.array(1), 20)
prediction_data.folder_to_dataset(prediction_hitting_folder, np.array(1), 30)
prediction_data.folder_to_dataset(prediction_movement_folder, np.array(1), 40)

In [10]:
predictions = loaded_model.predict(prediction_data.data)
predicted_classes = (predictions > 0.5).astype(int)
predicted_classes = np.squeeze(predicted_classes)
print("Predicted classes:", predicted_classes)

2024-05-07 19:06:37.824788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-05-07 19:06:38.007898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2/2 [==============================] - 2s 160ms/step
Predicted classes: [1 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 0 1 0 1 0]


In [44]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(prediction_data.labels, predicted_classes)
recall = recall_score(prediction_data.labels, predicted_classes)
f1 = f1_score(prediction_data.labels, predicted_classes)

print("適合率（Precision）: {:.2f}".format(precision))
print("再現率（Recall）: {:.2f}".format(recall))
print("F値: {:.2f}".format(f1))

適合率（Precision）: 0.95
再現率（Recall）: 0.95
F値: 0.95


# CNNで予測する

In [12]:
loaded_model = load_model('cnn_20240507_binary_model_best_v1.keras')

In [13]:
prediction_data = VariableDataSet(num_samples = 50, scale = 222)
prediction_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), 0, signal_processing='fft')
prediction_data.folder_to_dataset(prediction_voice_folder, np.array(1), 10, signal_processing='fft')
prediction_data.folder_to_dataset(prediction_cough_folder, np.array(1), 20, signal_processing='fft')
prediction_data.folder_to_dataset(prediction_hitting_folder, np.array(1), 30, signal_processing='fft')
prediction_data.folder_to_dataset(prediction_movement_folder, np.array(1), 40, signal_processing='fft')

In [14]:
predictions = loaded_model.predict(prediction_data.data)
predicted_classes = (predictions > 0.5).astype(int)
predicted_classes = np.squeeze(predicted_classes)
print("Predicted classes:", predicted_classes)

2/2 [==============================] - 0s 3ms/step
Predicted classes: [0 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 0 0 0 0 1 0]


# 真陽性と真陰性をカウント

In [11]:
result  = prediction_data.labels == predicted_classes
print(sum(result[:10]))
print(sum(result[10:]))

8
36
